# CB-Geo MPM 
> Krishna Kumar, UT Austin

## Run
This example shows how to run CB-Geo MPM.

In [1]:
# Define paths on DesignSafe
import os

def fetch_input_uri(cur_dir):
    if ('jupyter/MyData' in cur_dir ):
        cur_dir = cur_dir.split('MyData').pop() 
        storage_id = 'designsafe.storage.default'
        input_dir = ag.profiles.get()['username']+ cur_dir
        input_uri = 'agave://{}//{}'.format(storage_id,input_dir)
        input_uri = input_uri.replace(" ","%20")
    elif('jupyter/mydata' in cur_dir ):
        cur_dir = cur_dir.split('mydata').pop()
        storage_id = 'designsafe.storage.default'
        input_dir = ag.profiles.get()['username']+ cur_dir
        input_uri = 'agave://{}//{}'.format(storage_id,input_dir)
        input_uri = input_uri.replace(" ","%20")
    elif('jupyter/MyProjects' in cur_dir):
        cur_dir = cur_dir.split('MyProjects/').pop()
        PRJ = cur_dir.split('/')[0]
        qq = {"value.projectId": str(PRJ)}
        cur_dir = cur_dir.split(PRJ).pop()
        project_uuid = ag.meta.listMetadata(q=str(qq))[0]["uuid"]
        input_dir = cur_dir
        input_uri = 'agave://project-{}{}'.format(project_uuid,cur_dir)
        input_uri = input_uri.replace(" ","%20")
    elif('jupyter/CommunityData' in cur_dir):
        cur_dir = cur_dir.split('jupyter/CommunityData').pop() 
        input_dir = cur_dir
        input_uri = 'agave://designsafe.storage.community/{}'.format(input_dir)
        input_uri = input_uri.replace(" ","%20")
    return input_uri

In [2]:
# Configure job submission scripts
from agavepy.agave import Agave
ag = Agave.restore()
import os

# ---------------------------------------------------------------------------------
#Select tapis-app
app_name   = 'mpm'
#app_id    = 'opensees-docker-2.5.0.6248u11'  # if using old opensees-docker-2.5.0
app_id     = 'mpm-1.0.0'
storage_id = 'designsafe.storage.default'

# ---------------------------------------------------------------------------------
# Define control tapis-app variables
control_batchQueue       = 'skx-dev'
control_jobname          = 'mpm'
control_nodenumber       = '1'
control_processorsnumber = '1'
control_memorypernode    = '1'
control_maxRunTime       = '00:10:00'
    
# Set input directory
input_uri = 'agave://designsafe.storage.default//kks32/mpm-benchmarks/2d/uniaxial_stress' #fetch_input_uri(os.getcwd())
inputs = {"inputDirectory": [ input_uri ]}
input_filename = 'mpm.json'
parameters = {"inputfile" : [input_filename]}

# Identify folder with input file in DesignSafe
cur_dir = os.getcwd()

# ===============================================================================
# Define job description
app = ag.apps.get(appId=app_id)
job_description = {}
job_description["name"]              = (control_jobname)
job_description["appId"]             = (app_id)
job_description["batchQueue"]        = (control_batchQueue)
job_description["nodeCount"]         = int(control_nodenumber)
job_description["processorsPerNode"] = int(control_processorsnumber)
job_description["memoryPerNode"]     = (control_memorypernode)
job_description["maxRunTime"]        = control_maxRunTime
job_description["archive"]           = True
job_description["inputs"]            = inputs
job_description["parameters"]        = parameters
print(job_description)

{'name': 'mpm', 'appId': 'mpm-1.0.0', 'batchQueue': 'skx-dev', 'nodeCount': 1, 'processorsPerNode': 1, 'memoryPerNode': '1', 'maxRunTime': '00:10:00', 'archive': True, 'inputs': {'inputDirectory': ['agave://designsafe.storage.default//kks32/mpm-benchmarks/2d/uniaxial_stress']}, 'parameters': {'inputfile': ['mpm.json']}}


## Submit and monitor job status

In [3]:
# Submit and monitor job
import time
job = ag.jobs.submit(body=job_description)
print(" Job launched. Status provided below")
print(" Can also check in DesignSafe portal under - Workspace > Tools & Application > Job Status")

status = ag.jobs.getStatus(jobId=job["id"])["status"]
while status != "FINISHED":
    status = ag.jobs.getStatus(jobId=job["id"])["status"]
    print(f"Status: {status}")
    time.sleep(10)
    if status == "FAILED":
        break

 Job launched. Status provided below
 Can also check in DesignSafe portal under - Workspace > Tools & Application > Job Status
Status: PROCESSING_INPUTS
Status: STAGING_JOB
Status: QUEUED
Status: RUNNING
Status: RUNNING
Status: RUNNING
Status: RUNNING
Status: RUNNING
Status: RUNNING
Status: ARCHIVING
Status: FINISHED


## Job archive folder location

In [4]:
jobinfo = ag.jobs.get(jobId=job.id)
jobinfo.archivePath
user = jobinfo.archivePath.split('/', 1)[0]

In [5]:
jobinfo.archivePath

'kks32/archive/jobs/job-3f43f709-bc68-4c04-a576-67d5856e152d-007'

In [6]:
path = jobinfo.archivePath.replace(user,'/home/jupyter/MyData')
print(path)
os.chdir(path)

/home/jupyter/MyData/archive/jobs/job-3f43f709-bc68-4c04-a576-67d5856e152d-007


# Post processing

In [7]:
os.chdir('./uniaxial_stress/results/uniaxial-stress-2d-usf/')

In [8]:
%pwd

'/home/jupyter/MyData/archive/jobs/job-3f43f709-bc68-4c04-a576-67d5856e152d-007/uniaxial_stress/results/uniaxial-stress-2d-usf'

In [9]:
!pip3 install tables --quiet

To read a particles HDF5 data, for example `particles09.h5` at step 9:

In [10]:
# Read HDF5 data
# !pip3 install pandas
import pandas as pd
df = pd.read_hdf('particles09.h5', 'table')
# Print column headers
print(list(df))

['id', 'mass', 'volume', 'pressure', 'coord_x', 'coord_y', 'coord_z', 'displacement_x', 'displacement_y', 'displacement_z', 'nsize_x', 'nsize_y', 'nsize_z', 'velocity_x', 'velocity_y', 'velocity_z', 'stress_xx', 'stress_yy', 'stress_zz', 'tau_xy', 'tau_yz', 'tau_xz', 'strain_xx', 'strain_yy', 'strain_zz', 'gamma_xy', 'gamma_yz', 'gamma_xz', 'epsilon_v', 'cell_id', 'status', 'material_id', 'nstate_vars', 'svars_0', 'svars_1', 'svars_2', 'svars_3', 'svars_4', 'svars_5', 'svars_6', 'svars_7', 'svars_8', 'svars_9', 'svars_10', 'svars_11', 'svars_12', 'svars_13', 'svars_14', 'svars_15', 'svars_16', 'svars_17', 'svars_18', 'svars_19']


In [11]:
!pwd

/home/jupyter/MyData/archive/jobs/job-3f43f709-bc68-4c04-a576-67d5856e152d-007/uniaxial_stress/results/uniaxial-stress-2d-usf


In [12]:
# Print all velocities
print(df[['velocity_x', 'velocity_y','velocity_z']])

   velocity_x  velocity_y  velocity_z
0         0.0         0.0         0.0
1         0.0         0.0         0.0
2         0.0         0.0         0.0
3         0.0         0.0         0.0


## Extract results 
> Evaluate the maximum stress in YY direction should be -1 Pa.

In [13]:
print("Maximum σ_yy: {}".format(df['stress_yy'].max()))
print("Minimum σ_xx: {}".format(df['stress_xx'].min()))

Maximum σ_yy: -0.9999999999999999
Minimum σ_xx: 0.0
